In [1]:
import pandas as pd
from ome import Bacteria, Genome, Proteome, Ome

In [2]:
df = pd.read_csv('biss.csv', encoding='utf-8')
df.dropna(subset=['gen'], inplace=True)
df = df.sample(frac=1)
df.reset_index(drop=True, inplace=True)

In [3]:
df.shape

(40, 4)

In [4]:
all_gen, all_prot = {}, {}
corpus_gen, corpus_prot = [], []

for i in range(2):
    bact_name = df['name'][i]
    print(bact_name, df['gen'][i], df['prot'][i])
    
    genome = Genome(bact_name, df['gen'][i])
    genome.get_kmers()
    
    all_gen[bact_name] = genome.seq
    corpus_gen.append(' '.join(genome.kmers['all']))
    
    print('DONE\n')

Lactobacillus salivarius https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/001/011/095/GCF_001011095.1_ASM101109v1/GCF_001011095.1_ASM101109v1_genomic.fna.gz nan
DONE

Clostridium bolteae https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/002/234/575/GCF_002234575.2_ASM223457v2/GCF_002234575.2_ASM223457v2_genomic.fna.gz nan
DONE



In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=10, max_df=1000)
vectorizer.fit_transform(corpus_gen[:5])

MemoryError: 

In [ ]:
from gensim.models import FastText

model = FastText(vector_size=50, window=3, min_count=1)
corpus = []

for bact in all_gen:
    kmer_text = []
    for gen in all_gen[bact]:
        kmer_text.append(getKmers(all_gen[bact][gen], 10, 10)) #extend
    corpus.append(kmer_text)
    
model.build_vocab(sentences=corpus)
model.train(sentences=corpus, total_examples=len(corpus), epochs=20)

In [137]:
kmer_len_gen = 30
filename_gen = 'C://Users/User/Downloads/GCF_009721605.1_ASM972160v1_genomic.fna'

gen_dict = {rec.id: str(rec.seq) for rec in SeqIO.parse(filename_gen, 'fasta')}
kmers_gen = {k: list(getKmers(gen_dict[k], kmer_len_gen, 1)) for k in gen_dict}
kmers_gen['all'] = sum(list(kmers_gen.values()), [])
    
kmers_gen_counter = {k: Counter(v) for k,v in kmers_gen.items()}